In [2]:
import numpy as np
import pandas as pd
import seaborn as sns
import importanceMatrix
import os
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import scale
from matplotlib import pyplot as plt

proj_path = os.path.dirname(os.getcwd())
fig_path = proj_path + '/2_docs/LaTeX/Figures'

In [8]:
#Import dermatology data
col_names = np.array([
    'erythema', 'scaling', 'definite borders',
    'itching', 'koebner phenomenon', 'polygonal papules',
    'follicular papules', 'oral mucosal involvement', 'knee and elbow involvement',
    'scalp involvement', 'family history', 'melanin incontinence',
    'eosinophils in the infiltrate', 'PNL infiltrate', 'fibrosis of the papillary dermis',
    'exocytosis', 'acanthosis', 'hyperkeratosis',
    'parakeratosis', 'clubbing of the rete ridges', 'elongation of the rete ridges',
    'thinning of the suprapapillary epidermis', 'spongiform pustule', 'munro microabcess',
    'focal hypergranulosis', 'disappearance of the granular layer',
    'vacuolisation and damage of basal layer',
    'spongiosis', 'saw-tooth appearance of retes', 'follicular horn plug',
    'perifollicular parakeratosis', 'inflammatory monoluclear inflitrate',
    'band-like infiltrate',
    'Age', 'Class'
])
col_names = np.array([lab.capitalize() for lab in col_names])
feature_names = col_names[:-1]
class_names = np.array(['psoriasis', 'seboreic dermatitis', 'lichen planus',
                        'pityriasis rosea', 'cronic dermatitis', 'pityriasis rubra pilaris'])
class_names = np.array([lab.capitalize() for lab in class_names])
class_tags = np.arange(len(class_names)) + 1
data = pd.read_csv(proj_path+'/0_data/dermatology.data.csv', header=None, names=col_names)
skip_rows = data.Age == '?'
data = data[~skip_rows]
data.Age = np.array(data.Age, dtype=int)

In [51]:
#Import wine and iris
from sklearn.datasets import load_iris, load_wine

iris = load_iris()
feature_names = iris.feature_names
class_names = iris.target_names
class_tags = np.array([0, 1, 2])


wines = load_wine()
feature_names = wines.feature_names
class_names = wines.target_names
class_tags = np.array([0, 1, 2])

X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rf_clf.fit(X_train, y_train)


X = iris.data
y = iris.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)
rf_clf = RandomForestClassifier(random_state=42, n_estimators=100)
rf_clf.fit(X_train, y_train)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2]


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=None,
            oob_score=False, random_state=42, verbose=0, warm_start=False)

In [63]:
#Breast tissue data

breast_data = pd.read_excel(proj_path+'/0_data/BreastTissue.xls', sheet_name=1)
breast_data = breast_data.iloc[:,1:]

feature_names = breast_data.columns[1:]
class_names = np.unique(breast_data.Class)
class_tags = np.arange(len(class_names))

string_to_int = dict(zip(class_names, class_tags))
breast_data.loc[:,'Class'] = breast_data.apply(lambda r: string_to_int[r.Class], axis=1)

breast_data

,Class,I0,PA500,HFS,DA,Area,A/DA,Max IP,DR,P
0,1,524.794072,0.187448,0.032114,228.800228,6843.598481,29.910803,60.204880,220.737212,556.828334
1,1,330.000000,0.226893,0.265290,121.154201,3163.239472,26.109202,69.717361,99.084964,400.225776
2,1,551.879287,0.232478,0.063530,264.804935,11888.391827,44.894903,77.793297,253.785300,656.769449
3,1,380.000000,0.240855,0.286234,137.640111,5402.171180,39.248524,88.758446,105.198568,493.701814
4,1,362.831266,0.200713,0.244346,124.912559,3290.462446,26.342127,69.389389,103.866552,424.796503
5,1,389.872978,0.150098,0.097738,118.625814,2475.557078,20.868620,49.757149,107.686164,429.385788
6,1,290.455141,0.144164,0.053058,74.635067,1189.545213,15.938154,35.703331,65.541324,330.267293
7,1,275.677393,0.153938,0.187797,91.527893,1756.234837,19.187974,39.305183,82.658682,331.588302
8,1,470.000000,0.213105,0.225497,184.590057,8185.360837,44.343455,84.482483,164.122511,603.315715
9,1,423.000000,0.219562,0.261799,172.371241,6108.106297,35.435762,79.056351,153.172903,558.274515


In [75]:
#Students data
col_names = np.array([
    'Gender', 'Caste', 'Class X Percentage', 'Class XII Percentage', 'Internal Assessment Percentage',
    'End Semester Percentage', 'Whether the student has back or arrear papers', 'Marital Status',
    'Lived in Town or Village', 'Admission Category', 'Family Monthly Income', 'Family Size',
    'Father Qualification', 'Mother Qualification', 'Father Occupation', 'Mother Occupation',
    'Number of Friends', 'Study Hours', 'Student School attended at Class X level', 'Medium',
    'Home to College Travel Time', 'Class Attendance Percentage'
])
col_names = np.array([lab.capitalize() for lab in col_names])
students_data = pd.read_csv(proj_path+'/0_data/Student_performances.csv', header=None, names=col_names)
students_data = students_data.loc[:,col_names!='Marital status']

string_to_int_list = [
    {'M':0,'F':1}, {'G':0,'ST':1,'SC':2,'OBC':3,'MOBC':4}, {'Best':4,'Vg':3,'Good':2,'Pass':1,'Fail':0},
    {'Best':4,'Vg':3,'Good':2,'Pass':1,'Fail':0}, {'Best':4,'Vg':3,'Good':2,'Pass':1,'Fail':0},
    {'Best':4,'Vg':3,'Good':2,'Pass':1,'Fail':0}, {'Y':1,'N':0},
    {'T':1,'V':0}, {'Free':0,'Paid':1},
    {'Vh':4,'High':3,'Am':2,'Medium':1,'Low':0}, {'Large':2,'Average':1,'Small':0},
    {'Il':0,'Um':1,'10':2,'12':3,'Degree':4,'Pg':5}, {'Il':0,'Um':1,'10':2,'12':3,'Degree':4,'Pg':5},
    {'Service':0,'Business':1,'Retired':2,'Farmer':3,'Others':4},
    {'Service':0,'Business':1,'Retired':2,'Housewife':3,'Others':4},
    {'Large':2,'Average':1,'Small':0}, {'Good':2,'Average':1,'Poor':0},
    {'Govt':1,'Private':0}, {'Eng':0,'Asm':1,'Hin':2,'Ben':3},
    {'Large':2,'Average':1,'Small':0}, {'Good':2,'Average':1,'Poor':0}
]
for col_name,string_to_int in zip(students_data.columns, string_to_int_list):
    #print(col_name,string_to_int,
    #     students_data.apply(lambda r: string_to_int[r[col_name]], axis=1))
    students_data.loc[:,col_name] = students_data.apply(lambda r: string_to_int[r[col_name]], axis=1)

{'Best':4,'Vg':3,'Good':2,'Pass':1,'Fail':0}
    
feature_names = students_data.columns[students_data.columns!='End semester percentage']
class_names = np.array(['Fail', 'Pass', 'Good', 'Vg', 'Best'])
class_tags = np.arange(len(class_names))
print(feature_names, class_names, class_tags)

students_data

Index(['Gender', 'Caste', 'Class x percentage', 'Class xii percentage',
       'Internal assessment percentage',
       'Whether the student has back or arrear papers',
       'Lived in town or village', 'Admission category',
       'Family monthly income', 'Family size', 'Father qualification',
       'Mother qualification', 'Father occupation', 'Mother occupation',
       'Number of friends', 'Study hours',
       'Student school attended at class x level', 'Medium',
       'Home to college travel time', 'Class attendance percentage'],
      dtype='object') ['Fail' 'Pass' 'Good' 'Vg' 'Best'] [0 1 2 3 4]


,Gender,Caste,Class x percentage,Class xii percentage,Internal assessment percentage,End semester percentage,Whether the student has back or arrear papers,Lived in town or village,Admission category,Family monthly income,...,Father qualification,Mother qualification,Father occupation,Mother occupation,Number of friends,Study hours,Student school attended at class x level,Medium,Home to college travel time,Class attendance percentage
0,1,0,2,2,3,2,1,0,1,1,...,1,2,3,3,2,0,1,1,0,2
1,0,3,3,3,3,3,0,0,1,0,...,1,0,0,0,0,0,1,1,1,1
2,1,3,2,2,3,2,0,0,1,2,...,3,2,0,3,1,1,1,1,2,2
3,0,4,1,2,3,2,0,0,1,1,...,3,1,1,1,2,0,1,1,1,1
4,0,0,2,2,3,3,0,0,1,2,...,2,3,0,3,2,0,0,1,0,2
5,0,3,3,2,3,3,0,0,1,1,...,2,1,3,3,0,0,0,0,1,1
6,1,3,2,3,2,2,0,0,1,3,...,3,4,0,0,1,0,1,1,0,2
7,1,3,2,2,2,2,0,0,1,1,...,0,1,3,3,0,0,0,1,1,2
8,1,4,3,2,2,3,1,0,1,0,...,3,2,4,3,0,2,1,1,1,2
9,1,3,3,3,2,3,1,0,1,0,...,4,4,4,3,2,1,1,1,1,2


In [81]:
#Cars data

mtcars_data = pd.read_csv(proj_path+'/0_data/mtcars.csv')
mtcars_data = mtcars_data.iloc[:,1:]

class_names = np.unique(mtcars_data.gear)
feature_names = mtcars_data.columns[mtcars_data.columns!='gear']
class_tags = class_names

mtcars_data

,mpg,cyl,disp,hp,drat,wt,qsec,vs,am,gear,carb
0,21.0,6,160.0,110,3.90,2.620,16.46,0,1,4,4
1,21.0,6,160.0,110,3.90,2.875,17.02,0,1,4,4
2,22.8,4,108.0,93,3.85,2.320,18.61,1,1,4,1
3,21.4,6,258.0,110,3.08,3.215,19.44,1,0,3,1
4,18.7,8,360.0,175,3.15,3.440,17.02,0,0,3,2
5,18.1,6,225.0,105,2.76,3.460,20.22,1,0,3,1
6,14.3,8,360.0,245,3.21,3.570,15.84,0,0,3,4
7,24.4,4,146.7,62,3.69,3.190,20.00,1,0,4,2
8,22.8,4,140.8,95,3.92,3.150,22.90,1,0,4,2
9,19.2,6,167.6,123,3.92,3.440,18.30,1,0,4,4


In [88]:
#Wholesale customers data

sales_data = pd.read_csv(proj_path+'/0_data/Wholesale customers data.csv')

class_names = np.unique(sales_data.Region)
feature_names = sales_data.columns[sales_data.columns!='Region']
class_tags = class_names
#print(np.unique(sales_data.Region, return_counts=True))
sales_data

(array([1, 2, 3]), array([ 77,  47, 316]))


,Channel,Region,Fresh,Milk,Grocery,Frozen,Detergents_Paper,Delicassen
0,2,3,12669,9656,7561,214,2674,1338
1,2,3,7057,9810,9568,1762,3293,1776
2,2,3,6353,8808,7684,2405,3516,7844
3,1,3,13265,1196,4221,6404,507,1788
4,2,3,22615,5410,7198,3915,1777,5185
5,2,3,9413,8259,5126,666,1795,1451
6,2,3,12126,3199,6975,480,3140,545
7,2,3,7579,4956,9426,1669,3321,2566
8,1,3,5963,3648,6192,425,1716,750
9,2,3,6006,11093,18881,1159,7425,2098


In [85]:
#WiFi signals data
wifi_data = pd.read_excel(proj_path+'/0_data/WiFi_signals.xls')

class_names = np.unique(wifi_data.Room)
feature_names = wifi_data.columns[wifi_data.columns!='Room']
class_tags = class_names
#print(class_names, feature_names, np.unique(wifi_data.Room, return_counts=True))

wifi_data

[1 2 3 4] Index(['Signal 1', 'Signal 2', 'Signal 3', 'Signal 4', 'Signal 5', 'Signal 6',
       'Signal 7'],
      dtype='object') (array([1, 2, 3, 4]), array([500, 500, 500, 500]))


,Signal 1,Signal 2,Signal 3,Signal 4,Signal 5,Signal 6,Signal 7,Room
0,-64,-56,-61,-66,-71,-82,-81,1
1,-68,-57,-61,-65,-71,-85,-85,1
2,-63,-60,-60,-67,-76,-85,-84,1
3,-61,-60,-68,-62,-77,-90,-80,1
4,-63,-65,-60,-63,-77,-81,-87,1
5,-64,-55,-63,-66,-76,-88,-83,1
6,-65,-61,-65,-67,-69,-87,-84,1
7,-61,-63,-58,-66,-74,-87,-82,1
8,-65,-60,-59,-63,-76,-86,-82,1
9,-62,-60,-66,-68,-80,-86,-91,1


In [90]:
#Import flag data
col_names = np.array([
    'name', 'landmass', 'zone', 'area', 'population', 'language', 'religion', 'bars', 'stripes', 'colours',
    'red', 'green', 'blue', 'gold', 'white', 'black', 'orange', 'mainhue', 'circles', 'crosses', 'saltires',
    'quarters', 'sunstars', 'crescent', 'triangle', 'icon', 'animate', 'text', 'topleft', 'botright'
])
col_names = np.array([lab.capitalize() for lab in col_names])
feature_names = col_names[col_names!='Religion']
class_names = np.array(['Catholic', 'Other Christian', 'Muslim', 'Buddhist', 'Hindu', 'Ethnic', 'Marxist', 'Others'])
class_tags = np.arange(len(class_names))
flag_data = pd.read_csv(proj_path+'/0_data/flag.data.csv', header=None, names=col_names)

print(class_names, feature_names, np.unique(flag_data.Religion, return_counts=True))

flag_data

['Catholic' 'Other Christian' 'Muslim' 'Buddhist' 'Hindu' 'Ethnic'
 'Marxist' 'Others'] ['Name' 'Landmass' 'Zone' 'Area' 'Population' 'Language' 'Bars' 'Stripes'
 'Colours' 'Red' 'Green' 'Blue' 'Gold' 'White' 'Black' 'Orange' 'Mainhue'
 'Circles' 'Crosses' 'Saltires' 'Quarters' 'Sunstars' 'Crescent'
 'Triangle' 'Icon' 'Animate' 'Text' 'Topleft' 'Botright'] (array([0, 1, 2, 3, 4, 5, 6, 7]), array([40, 60, 36,  8,  4, 27, 15,  4]))


,Name,Landmass,Zone,Area,Population,Language,Religion,Bars,Stripes,Colours,...,Saltires,Quarters,Sunstars,Crescent,Triangle,Icon,Animate,Text,Topleft,Botright
0,Afghanistan,5,1,648,16,10,2,0,3,5,...,0,0,1,0,0,1,0,0,black,green
1,Albania,3,1,29,3,6,6,0,0,3,...,0,0,1,0,0,0,1,0,red,red
2,Algeria,4,1,2388,20,8,2,2,0,3,...,0,0,1,1,0,0,0,0,green,white
3,American-Samoa,6,3,0,0,1,1,0,0,5,...,0,0,0,0,1,1,1,0,blue,red
4,Andorra,3,1,0,0,6,0,3,0,3,...,0,0,0,0,0,0,0,0,blue,red
5,Angola,4,2,1247,7,10,5,0,2,3,...,0,0,1,0,0,1,0,0,red,black
6,Anguilla,1,4,0,0,1,1,0,1,3,...,0,0,0,0,0,0,1,0,white,blue
7,Antigua-Barbuda,1,4,0,0,1,1,0,1,5,...,0,0,1,0,1,0,0,0,black,red
8,Argentina,2,3,2777,28,2,0,0,3,2,...,0,0,0,0,0,0,0,0,blue,blue
9,Argentine,2,3,2777,28,2,0,0,3,3,...,0,0,1,0,0,0,0,0,blue,blue


In [95]:
#Import tumor data
col_names = np.array([
    'class', 'age', 'sex', 'histologic-type', 'degree-of-diffe', 'bone', 'bone-marrow', 'lung', 'pleura',
    'peritoneum', 'liver', 'brain', 'skin', 'neck', 'supraclavicular', 'axillar', 'mediastinum', 'abdominal'
])
col_names = np.array([lab.capitalize() for lab in col_names])
feature_names = col_names[col_names!='Class']
class_names = np.array([
    'lung', 'head & neck', 'esophasus', 'thyroid', 'stomach', 'duoden & sm.int',
    'colon', 'rectum', 'anus', 'salivary glands', 'pancreas', 'gallblader',
    'liver', 'kidney', 'bladder', 'testis', 'prostate', 'ovary', 'corpus uteri', 
    'cervix uteri', 'vagina', 'breast'
])
#class_tags = np.arange(len(class_names))
tumor_data = pd.read_csv(proj_path+'/0_data/primary-tumor.data.csv', header=None, names=col_names)
tumor_data = tumor_data.loc[:,[not(col in ['Histologic-type', 'Degree-of-diffe']) for col in col_names]]
tumor_data.apply(lambda r: any(r=='?'),axis=1)
tumor_data = tumor_data.loc[~(tumor_data.apply(lambda r: any(r=='?'),axis=1)),:]
low_data_classes = np.array([6, 9, 10, 15, 16, 20, 21])
print('remove classes', class_names[low_data_classes-1])
tumor_data = tumor_data.loc[~tumor_data.Class.isin(low_data_classes),:]

class_names = np.array([col for col in class_names if not(col in class_names[low_data_classes-1])])
class_tags = np.unique(tumor_data.Class)
print(class_names, feature_names, np.unique(tumor_data.Class, return_counts=True))

tumor_data

remove classes ['duoden & sm.int' 'anus' 'salivary glands' 'bladder' 'testis'
 'cervix uteri' 'vagina']
['lung' 'head & neck' 'esophasus' 'thyroid' 'stomach' 'colon' 'rectum'
 'pancreas' 'gallblader' 'liver' 'kidney' 'prostate' 'ovary'
 'corpus uteri' 'breast'] ['Age' 'Sex' 'Histologic-type' 'Degree-of-diffe' 'Bone' 'Bone-marrow'
 'Lung' 'Pleura' 'Peritoneum' 'Liver' 'Brain' 'Skin' 'Neck'
 'Supraclavicular' 'Axillar' 'Mediastinum' 'Abdominal'] (array([ 1,  2,  3,  4,  5,  7,  8, 11, 12, 13, 14, 17, 18, 19, 22]), array([82, 20,  9, 13, 39, 14,  6, 28, 16,  7, 24, 10, 29,  6, 24]))


,Class,Age,Sex,Bone,Bone-marrow,Lung,Pleura,Peritoneum,Liver,Brain,Skin,Neck,Supraclavicular,Axillar,Mediastinum,Abdominal
0,1,1,1,2,2,1,2,2,2,2,2,2,2,2,2,2
1,1,1,1,2,2,2,2,2,1,2,2,2,1,2,1,2
2,1,1,2,1,2,2,2,2,2,2,2,2,2,2,1,2
3,1,1,2,1,2,1,1,2,2,2,2,2,2,2,1,2
4,1,1,2,1,2,1,1,2,2,2,2,2,2,2,1,2
5,1,1,2,1,2,2,2,2,2,1,2,2,1,1,1,2
6,1,2,1,1,2,2,2,2,2,2,1,1,1,2,2,2
7,1,2,1,1,2,2,2,2,2,2,2,2,2,2,2,2
8,1,2,1,2,2,2,2,2,2,2,2,1,2,2,2,2
10,1,2,1,1,2,2,1,2,2,2,2,2,1,2,1,2


In [ ]:
X = np.array(data.loc[:,col_names[:-1]])
y = np.array(data.loc[:,col_names[-1]])
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=45)

# train the classifier
rf_clf = RandomForestClassifier(random_state=45, n_estimators=100)
rf_clf.fit(X_train, y_train)

# make predictions and calculate the MAE error
predictions = rf_clf.predict(X_test)
errors = abs(predictions - y_test)
print('Mean Absolute Error: {}'.format(round(np.mean(errors), 2))) 

# calculate the accuracy of the classifier
acc = 100 * accuracy_score(y_test, predictions)
print('Model Accuracy: {}'.format(round(acc, 2)))

print(np.unique(y, return_counts=True))
print(np.unique(y_train, return_counts=True))
print(np.unique(y_test, return_counts=True))

In [ ]:
###Class count plot
fig, ax = plt.subplots(figsize=(3.24*1.5, 2*1.5))
sns.countplot(y, color=sns.color_palette()[0], ax=ax, orient='h')
ax.set_ylabel('Count')
ax.set_xlabel('Class')
ax.set_xticklabels(np.unique(y), rotation=45, ha='right', rotation_mode='anchor')
sns.despine(fig=fig)
#fig.savefig(fig_path+'/class_counts.pdf', bbox_inches='tight')

In [ ]:
global_importance = rf_clf.feature_importances_
global_index = np.argsort(global_importance)

imp_mat = importanceMatrix.calcImportanceMatrix(rf_clf)
mean_importance = np.mean(imp_mat, axis=0)
pca_index = np.argsort(mean_importance)

unique_classes, count_classes = np.unique(y, return_counts=True)
rare_class_ind = np.argsort(count_classes)[0]
rare_class = unique_classes[rare_class_ind]
rare_class_index = np.argsort(imp_mat[rare_class_ind])

top5_global_features = feature_names[global_index[-5:]]
top5_pca_features = feature_names[pca_index[-5:]]
top5_rare_features = feature_names[rare_class_index[-5:]]
print(top5_global_features)
print(top5_pca_features)
print(top5_rare_features)

In [ ]:
def plot_confusion_matrix(y_true, y_pred, classes,
                          title='Normalized confusion matrix',
                          cmap=plt.cm.plasma):
    # Compute confusion matrix
    cm = confusion_matrix(y_true, y_pred)
    cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis] * 100
    #print(cm)
    fig, ax = plt.subplots()
    im = ax.imshow(cm, interpolation='nearest', cmap=cmap)
    ax.figure.colorbar(im, ax=ax, label='Percentage')
    # We want to show all ticks...
    ax.set(xticks=np.arange(cm.shape[1]),
           yticks=np.arange(cm.shape[0]),
           # ... and label them with the respective list entries
           xticklabels=classes, yticklabels=classes,
           title=title,
           ylabel='True label',
           xlabel='Predicted label')

    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right",
             rotation_mode="anchor")

    # Loop over data dimensions and create text annotations.
    fmt = '.0f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(round(cm[i, j],0), fmt),
                    ha="center", va="center",
                    color="black" if cm[i, j] > thresh else "white")
    plt.tight_layout()
    return fig, ax

In [ ]:
feat_ind = np.array([feat in top5_global_features for feat in feature_names])
X_train, X_test, y_train, y_test = train_test_split(X[:,feat_ind], y, random_state=45)
# train the classifier
rf_clf = RandomForestClassifier(random_state=45, n_estimators=100)
rf_clf.fit(X_train, y_train)
# make predictions and calculate the MAE error
predictions = rf_clf.predict(X_test)
errors = abs(predictions - y_test)
print('Mean Absolute Error: {}'.format(round(np.mean(errors), 2))) 
# calculate the accuracy of the classifier
acc = 100 * accuracy_score(y_test, predictions)
print('Model Accuracy: {}'.format(round(acc, 2)))

fig, ax = plot_confusion_matrix(y_test, predictions, class_names,
                      title='Default method')
fig.savefig(fig_path+'/confusion_matrix_default_method.pdf', bbox_inches='tight')
plt.show()

In [ ]:
feat_ind = np.array([feat in top5_pca_features for feat in feature_names])
X_train, X_test, y_train, y_test = train_test_split(X[:,feat_ind], y, random_state=45)
# train the classifier
rf_clf = RandomForestClassifier(random_state=45, n_estimators=100)
rf_clf.fit(X_train, y_train)
# make predictions and calculate the MAE error
predictions = rf_clf.predict(X_test)
errors = abs(predictions - y_test)
print('Mean Absolute Error: {}'.format(round(np.mean(errors), 2))) 
# calculate the accuracy of the classifier
acc = 100 * accuracy_score(y_test, predictions)
print('Model Accuracy: {}'.format(round(acc, 2)))

fig, ax = plot_confusion_matrix(y_test, predictions, class_names,
                      title='Per-class average method')
fig.savefig(fig_path+'/confusion_matrix_perclass_average.pdf', bbox_inches='tight')
plt.show()

In [ ]:
feat_ind = np.array([feat in top5_rare_features for feat in feature_names])
X_train, X_test, y_train, y_test = train_test_split(X[:,feat_ind], y, random_state=45)
# train the classifier
rf_clf = RandomForestClassifier(random_state=45, n_estimators=100)
rf_clf.fit(X_train, y_train)
# make predictions and calculate the MAE error
predictions = rf_clf.predict(X_test)
errors = abs(predictions - y_test)
print('Mean Absolute Error: {}'.format(round(np.mean(errors), 2))) 
# calculate the accuracy of the classifier
acc = 100 * accuracy_score(y_test, predictions)
print('Model Accuracy: {}'.format(round(acc, 2)))

fig, ax = plot_confusion_matrix(y_test, predictions, class_names,
                      title='Rare class method')
fig.savefig(fig_path+'/confusion_matrix_rare_class.pdf', bbox_inches='tight')
plt.show()